In [78]:
import pandas as pd
import numpy as np
import scipy.io
import mat73
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings
import os
# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)


## EEG data:

In [77]:
 ## data loading
data = mat73.loadmat('x_source.mat')  
id = scipy.io.loadmat('id.mat')
IDs = pd.DataFrame(id["id"])
#flattening and adding id
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2['IDs'] = IDs


## Behavioral Data:


In [38]:
behaviour_data = pd.read_csv('scires.csv', low_memory=False)
list(behaviour_data.columns)

['Unnamed: 0',
 'IDs',
 'Age',
 'Sex',
 'ARI_P_Total_Score',
 'ARI_S_Total_Score',
 'MFQ_P_Total',
 'MFQ_SR_Total',
 'BMI.1',
 'HeartRate',
 'Height',
 'Weight',
 'SCARED_P_Total',
 'SCARED_SR_Total',
 'SRS_AWR',
 'SRS_COG',
 'SRS_COM',
 'SRS_DSMRRB',
 'SRS_MOT',
 'SRS_RRB',
 'SRS_SCI',
 'SRS_Total',
 'SWAN_HY',
 'SWAN_IN',
 'SWAN_Total',
 'WISC_FSIQ',
 'DX_01',
 'DX_01_ByHx',
 'DX_01_Cat',
 'DX_01_Code',
 'DX_01_Confirmed',
 'DX_01_New',
 'DX_01_PRem',
 'DX_01_Past_Doc',
 'DX_01_Presum',
 'DX_01_RC',
 'DX_01_Rem',
 'DX_01_RuleOut',
 'DX_01_Spec',
 'DX_01_Sub',
 'DX_01_Time',
 'DX_02',
 'DX_02_ByHx',
 'DX_02_Cat',
 'DX_02_Code',
 'DX_02_Confirmed',
 'DX_02_New',
 'DX_02_PRem',
 'DX_02_Past_Doc',
 'DX_02_Presum',
 'DX_02_RC',
 'DX_02_Rem',
 'DX_02_RuleOut',
 'DX_02_Spec',
 'DX_02_Sub',
 'DX_02_Time',
 'DX_03',
 'DX_03_ByHx',
 'DX_03_Cat',
 'DX_03_Code',
 'DX_03_Confirmed',
 'DX_03_New',
 'DX_03_PRem',
 'DX_03_Past_Doc',
 'DX_03_Presum',
 'DX_03_RC',
 'DX_03_Rem',
 'DX_03_RuleOut',
 'DX_

### We need columns with:
1. IDs

 
2. for Anxiety:  "SCARED_SR" "SCARED_P". 
    


2. for ADHD inattention: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN IN")


3. for ADHD combined: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN HY")


4. for Autism Social Responsiveness Scale-2 ("SRS_Pre" (preschool) or "SRS") 


5. for Depression: Children's Depression Index (average score of: "CDI_SR" "CDI_P") (alternative 1: Mood & Feelings Questionnaire (MFQ); Alternative 2: Affective Reactivity Index—(ARI-S) Self Report)


6. WISC-Full IQ "WISC_FSIQ"


In [79]:
df = behaviour_data[['IDs', 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]



In [80]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


missing_values_table(df)

Your selected dataframe has 8 columns (with column 'IDs').
There are 7 columns that have missing values.


,Missing Values,% of Total Values
WISC_FSIQ,588,19.1
MFQ_P_Total,466,15.1
SCARED_P_Total,328,10.7
SRS_Total,218,7.1
SWAN_HY,163,5.3
SWAN_IN,163,5.3
ARI_S_Total_Score,125,4.1
